In [1]:

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
#scripts
from src import rank_metrics
from src import helpers
from src import table_encoder
from src import metrics
    
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql

from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer



In [2]:
data_path = '../data/csv'

In [3]:
ratings_df, movies_df, encoded_movies_df, tags_df, enoded_tags_df = helpers.load_format_data(data_path)

In [4]:
train, test = helpers.load_format_data_for_model(data_path)

In [5]:
model = helpers.fit_als(train)

In [25]:
allUserRecs, allMovieRecs = helpers.predict_for_all_users(model, test)

RMSE = 1.0801361401777707


In [26]:
user_prediction = helpers.predict_for_one_user(model, 5, test, movies_df)
user_prediction

['Toy Story (1995)',
 'Once Were Warriors (1994)',
 'Apollo 13 (1995)',
 'Stargate (1994)',
 'Batman (1989)']

In [27]:
user_liked_movie_ids = helpers.get_user_liked_movie_ids(5, ratings_df)

In [28]:
user_liked_movie_titles = helpers.get_user_liked_movie_titles(5, ratings_df, movies_df)[:5]

In [29]:
rec_movie_ids = helpers.get_ids_from_titles(user_prediction, movies_df)
rec_movie_ids

[1, 290, 150, 316, 592]

In [30]:
metrics.compare_recs_to_user_likes(rec_movie_ids, user_liked_movie_ids, encoded_genres=encoded_movies_df)

,movie ids,similarity
0,1,0.313626
1,290,0.213679
2,150,0.228601
3,316,0.292763
4,592,0.256397


In [31]:
metrics.compare_movie(232, 984, encoded_movies_df)

0.8165

In [32]:
diction = {'blast':5,
          'cop':0.5,
          'toys':0}

In [82]:
tits = list(diction.keys())
rats = list(diction.values())
us = [9999]*len(tits)
movieIds = []
for title in tits:
    movieIds.append(movies_df[movies_df['title']==title]['movieId'])

In [83]:
movieIds = pd.Series(movieIds)
rats = pd.Series(rats)
us = pd.Series(us)

In [84]:
df = pd.DataFrame()

In [90]:
movies_df[movies_df['title']=='Toy Story (1995)']['movieId']

0    1
Name: movieId, dtype: int64

In [85]:
df['userId'] = us
df['movieId'] = movieIds
df['rating'] = rats


In [86]:
df

,movieTitle,rating,userId
0,blast,5.0,9999
1,cop,0.5,9999
2,toys,0.0,9999


In [74]:
us, rats, tits

Empty DataFrame
Columns: [(9999, 5.0, blast), (9999, 0.5, cop), (9999, 0.0, toys)]
Index: []